In [1]:
# !pip install opencv-python

In [2]:
def webcamTest():
    import cv2

    cap = cv2.VideoCapture(0)
    cap.set(3, 640)
    cap.set(4, 480)

    while True:
        ret, img= cap.read()
        cv2.imshow('Webcam', img)

        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [3]:
# !pip3 install ultralytics

In [3]:
from ultralytics import YOLO
import cv2
import math 
import time

# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# model
model = YOLO("yolo-Weights/yolov8n.pt")

# object classes
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

# person 클래스의 인덱스를 찾습니다.
person_index = classNames.index('person')

# 중심점의 히스토리를 저장할 리스트
center_points_history = []

while True:
    success, img = cap.read()
    results = model(img, stream=True)

    # 현재 시간
    current_time = time.time()

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # class name
            cls = int(box.cls[0])

            if cls == person_index:  # person 클래스만 확인
                # bounding box
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # 중심점 계산
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)

                # 중심점의 좌표와 현재 시간을 히스토리에 추가
                center_points_history.append(((center_x, center_y), current_time))

                # 바운딩 박스 그리기
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

                # confidence
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confidence --->", confidence)

                # object details
                org = [x1, y1]
                font = cv2.FONT_HERSHEY_SIMPLEX
                fontScale = 1
                color = (255, 0, 0)
                thickness = 2

                cv2.putText(img, str(classNames[cls]), org, font, fontScale, color, thickness)

    # 0.3초 이전의 중심점들을 히스토리에서 제거
    center_points_history = [(point, timestamp) for point, timestamp in center_points_history if current_time - timestamp <= 0.3]

    # 중심점들을 이미지에 그리기
    for idx, ((center_x, center_y), _) in enumerate(center_points_history):
        # 가장 최근의 점만 붉은색으로 그리기
        if idx == len(center_points_history) - 1:
            cv2.circle(img, (center_x, center_y), 3, (0, 0, 255), -1)  # RGB로 (0, 0, 255)는 붉은색입니다.
        else:
            cv2.circle(img, (center_x, center_y), 3, (0, 255, 0), -1)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 1 person, 1 chair, 1 tv, 1 laptop, 1 clock, 78.6ms
Speed: 2.4ms preprocess, 78.6ms inference, 11.2ms postprocess per image at shape (1, 3, 480, 640)
2023-11-03 18:45:15.445 Python[34326:428678] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.

0: 480x640 1 person, 2 chairs, 1 tv, 1 laptop, 1 clock, 59.3ms
Speed: 0.8ms preprocess, 59.3ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)


Confidence ---> 0.84
Confidence ---> 0.87



0: 480x640 1 person, 2 chairs, 1 tv, 2 laptops, 1 clock, 62.9ms
Speed: 1.0ms preprocess, 62.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 5 chairs, 1 tv, 1 laptop, 1 clock, 56.9ms
Speed: 1.1ms preprocess, 56.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 bowl, 3 chairs, 1 tv, 2 laptops, 1 clock, 56.9ms
Speed: 1.0ms preprocess, 56.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)


Confidence ---> 0.84
Confidence ---> 0.85
Confidence ---> 0.85



0: 480x640 1 person, 3 chairs, 1 tv, 1 laptop, 1 clock, 60.5ms
Speed: 1.1ms preprocess, 60.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 2 chairs, 1 tv, 2 laptops, 1 clock, 55.9ms
Speed: 1.3ms preprocess, 55.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 4 chairs, 1 tv, 2 laptops, 1 clock, 55.0ms
Speed: 1.2ms preprocess, 55.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)


...

Confidence ---> 0.81
Confidence ---> 0.81
Confidence ---> 0.83



0: 480x640 1 person, 2 chairs, 1 tv, 2 laptops, 1 clock, 55.6ms
Speed: 0.9ms preprocess, 55.6ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 3 chairs, 1 tv, 1 laptop, 1 clock, 73.6ms
Speed: 3.4ms preprocess, 73.6ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)



Confidence ---> 0.8
Confidence ---> 0.8


KeyboardInterrupt: 

: 